# Importing libraries

In [5]:
import pandas as pd
import numpy as np
from sklearn import preprocessing as prp
from pandas.api.types import is_numeric_dtype

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
pd.set_option('display.max_columns', None)

# Data Pre-processing


In [7]:
dataset = pd.read_csv('/home/sankarshan/Projects/MLProject/music-recommendation-app/Datasets/dataset.csv')

In [8]:
dataset.head()

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,1,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,1,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,0,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,0,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,2,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic


In [9]:
dataset.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114000 entries, 0 to 113999
Data columns (total 21 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Unnamed: 0        114000 non-null  int64  
 1   track_id          114000 non-null  object 
 2   artists           113999 non-null  object 
 3   album_name        113999 non-null  object 
 4   track_name        113999 non-null  object 
 5   popularity        114000 non-null  int64  
 6   duration_ms       114000 non-null  int64  
 7   explicit          114000 non-null  bool   
 8   danceability      114000 non-null  float64
 9   energy            114000 non-null  float64
 10  key               114000 non-null  int64  
 11  loudness          114000 non-null  float64
 12  mode              114000 non-null  int64  
 13  speechiness       114000 non-null  float64
 14  acousticness      114000 non-null  float64
 15  instrumentalness  114000 non-null  float64
 16  liveness          11

 **Usefull Features**


---


*   track_id
*   artists
*   popularity
*   danceability
*   energy
*   speechiness
*   acousticness
*   liveness
*   valence
*   track_genre
*   tempo
*   key
*   mode









In [10]:
dataset = dataset[['track_id', 'artists', 'track_genre', 'mode' , 'key' ,'popularity' ,'danceability', 'energy', 'speechiness', 'acousticness', 'liveness', 'valence', 'tempo' ]]
dataset.head()

,track_id,artists,track_genre,mode,key,popularity,danceability,energy,speechiness,acousticness,liveness,valence,tempo
0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,acoustic,0,1,73,0.676,0.4610,0.1430,0.0322,0.3580,0.715,87.917
1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,acoustic,1,1,55,0.420,0.1660,0.0763,0.9240,0.1010,0.267,77.489
2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,acoustic,1,0,57,0.438,0.3590,0.0557,0.2100,0.1170,0.120,76.332
3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,acoustic,1,0,71,0.266,0.0596,0.0363,0.9050,0.1320,0.143,181.740
4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,acoustic,1,2,82,0.618,0.4430,0.0526,0.4690,0.0829,0.167,119.949


In [11]:

print(len(pd.unique(dataset['track_id'])))
print(len(pd.unique(dataset['track_genre'])))
print(dataset['track_genre'].unique())


89741
114
['acoustic' 'afrobeat' 'alt-rock' 'alternative' 'ambient' 'anime'
 'black-metal' 'bluegrass' 'blues' 'brazil' 'breakbeat' 'british'
 'cantopop' 'chicago-house' 'children' 'chill' 'classical' 'club' 'comedy'
 'country' 'dance' 'dancehall' 'death-metal' 'deep-house' 'detroit-techno'
 'disco' 'disney' 'drum-and-bass' 'dub' 'dubstep' 'edm' 'electro'
 'electronic' 'emo' 'folk' 'forro' 'french' 'funk' 'garage' 'german'
 'gospel' 'goth' 'grindcore' 'groove' 'grunge' 'guitar' 'happy'
 'hard-rock' 'hardcore' 'hardstyle' 'heavy-metal' 'hip-hop' 'honky-tonk'
 'house' 'idm' 'indian' 'indie-pop' 'indie' 'industrial' 'iranian'
 'j-dance' 'j-idol' 'j-pop' 'j-rock' 'jazz' 'k-pop' 'kids' 'latin'
 'latino' 'malay' 'mandopop' 'metal' 'metalcore' 'minimal-techno' 'mpb'
 'new-age' 'opera' 'pagode' 'party' 'piano' 'pop-film' 'pop' 'power-pop'
 'progressive-house' 'psych-rock' 'punk-rock' 'punk' 'r-n-b' 'reggae'
 'reggaeton' 'rock-n-roll' 'rock' 'rockabilly' 'romance' 'sad' 'salsa'
 'samba' 'sertan

In [12]:
dataset.drop_duplicates(subset=['track_id'], inplace = True)


In [13]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 89741 entries, 0 to 113999
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   track_id      89741 non-null  object 
 1   artists       89740 non-null  object 
 2   track_genre   89741 non-null  object 
 3   mode          89741 non-null  int64  
 4   key           89741 non-null  int64  
 5   popularity    89741 non-null  int64  
 6   danceability  89741 non-null  float64
 7   energy        89741 non-null  float64
 8   speechiness   89741 non-null  float64
 9   acousticness  89741 non-null  float64
 10  liveness      89741 non-null  float64
 11  valence       89741 non-null  float64
 12  tempo         89741 non-null  float64
dtypes: float64(7), int64(3), object(3)
memory usage: 9.6+ MB


In [14]:
dataset.dropna(inplace = True)

In [15]:
dataset.reset_index(drop = True, inplace = True)


## Generate 'artists' list

In [16]:
def generate_list(col_name):
  return list(dataset[col_name].apply(lambda x : [i.replace("-", "").replace(" ", "").replace(".","") for i in x]))

In [17]:
dataset['artists'] = dataset['artists'].apply(lambda x:x.lower().split(';'))
  

In [18]:
dataset['artists'] = generate_list('artists')

## Generate 'track_genres' list

In [19]:
dataset['track_genre'] = dataset['track_genre'].apply(lambda x: x.split())

In [20]:
dataset['track_genre'] = generate_list('track_genre')

# One Hot Encoding 


In [21]:
def ohe_prep(df, column, new_name): 
    
    tf_df = pd.get_dummies(df[column])
    feature_names = tf_df.columns
    tf_df.columns = [new_name + "|" + str(i) for i in feature_names]
    tf_df.reset_index(drop = True, inplace = True)    
    return tf_df

# Feature generation

In [22]:

def create_feature_set(df, float_cols):
    
    tfidf = TfidfVectorizer()
    tfidf_matrix =  tfidf.fit_transform(dataset['track_genre'].apply(lambda x: " ".join(x)))
    genre_df = pd.DataFrame(tfidf_matrix.toarray())
    genre_df.columns = ['genre' + "|" + i for i in tfidf.get_feature_names_out()]
    genre_df.reset_index(drop = True, inplace=True)

    key_ohe = ohe_prep(dataset, 'key','key') * 0.5
    mode_ohe = ohe_prep(dataset, 'mode','mode') * 0.5

    pop = dataset[['popularity']].reset_index(drop = True)
    scaler = MinMaxScaler()
    pop_scaled = pd.DataFrame(scaler.fit_transform(pop), columns = pop.columns) * 0.2 

    floats = dataset[float_cols].reset_index(drop = True)
    scaler = MinMaxScaler()
    floats_scaled = pd.DataFrame(scaler.fit_transform(floats), columns = floats.columns) * 0.2

    final = pd.concat([genre_df,floats_scaled, pop_scaled, key_ohe, mode_ohe], axis = 1)

    final['track_id']=dataset['track_id'].values
    
    return final

In [23]:
float_cols = dataset.dtypes[dataset.dtypes == 'float64'].index.values


In [24]:
complete_feature_set = create_feature_set(dataset, float_cols=float_cols)
complete_feature_set

,genre|acoustic,genre|afrobeat,genre|alternative,genre|altrock,genre|ambient,genre|anime,genre|blackmetal,genre|bluegrass,genre|blues,genre|brazil,genre|breakbeat,genre|british,genre|cantopop,genre|chicagohouse,genre|children,genre|chill,genre|classical,genre|club,genre|comedy,genre|country,genre|dance,genre|dancehall,genre|deathmetal,genre|deephouse,genre|detroittechno,genre|disco,genre|disney,genre|drumandbass,genre|dub,genre|dubstep,genre|edm,genre|electro,genre|electronic,genre|emo,genre|folk,genre|forro,genre|french,genre|funk,genre|garage,genre|german,genre|gospel,genre|goth,genre|grindcore,genre|groove,genre|grunge,genre|guitar,genre|happy,genre|hardcore,genre|hardrock,genre|hardstyle,genre|heavymetal,genre|hiphop,genre|honkytonk,genre|house,genre|idm,genre|indian,genre|indie,genre|indiepop,genre|industrial,genre|iranian,genre|jazz,genre|jdance,genre|jidol,genre|jpop,genre|jrock,genre|kids,genre|kpop,genre|latin,genre|latino,genre|malay,genre|mandopop,genre|metal,genre|metalcore,genre|minimaltechno,genre|mpb,genre|newage,genre|opera,genre|pagode,genre|party,genre|piano,genre|pop,genre|popfilm,genre|powerpop,genre|progressivehouse,genre|psychrock,genre|punk,genre|punkrock,genre|reggae,genre|reggaeton,genre|rnb,genre|rock,genre|rockabilly,genre|rocknroll,genre|romance,genre|sad,genre|salsa,genre|samba,genre|sertanejo,genre|showtunes,genre|singersongwriter,genre|ska,genre|sleep,genre|soul,genre|spanish,genre|study,genre|swedish,genre|synthpop,genre|tango,genre|techno,genre|trance,genre|triphop,genre|turkish,genre|worldmusic,danceability,energy,speechiness,acousticness,liveness,valence,tempo,popularity,key|0,key|1,key|2,key|3,key|4,key|5,key|6,key|7,key|8,key|9,key|10,key|11,mode|0,mode|1,track_id
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.137259,0.09220,0.029637,0.006466,0.07160,0.143719,0.072249,0.146,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,5SuOikwiRyPMVoIQDJUgSV
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.085279,0.03320,0.015813,0.185542,0.02020,0.053668,0.063679,0.110,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,4qPNDBW1i3p13qLCt0Ki3A
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.088934,0.07180,0.011544,0.042169,0.02340,0.024121,0.062729,0.114,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,1iJBSr7s7jYXzM8EGcbK5b
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.054010,0.01192

In [25]:
complete_feature_set.head()

,genre|acoustic,genre|afrobeat,genre|alternative,genre|altrock,genre|ambient,genre|anime,genre|blackmetal,genre|bluegrass,genre|blues,genre|brazil,genre|breakbeat,genre|british,genre|cantopop,genre|chicagohouse,genre|children,genre|chill,genre|classical,genre|club,genre|comedy,genre|country,genre|dance,genre|dancehall,genre|deathmetal,genre|deephouse,genre|detroittechno,genre|disco,genre|disney,genre|drumandbass,genre|dub,genre|dubstep,genre|edm,genre|electro,genre|electronic,genre|emo,genre|folk,genre|forro,genre|french,genre|funk,genre|garage,genre|german,genre|gospel,genre|goth,genre|grindcore,genre|groove,genre|grunge,genre|guitar,genre|happy,genre|hardcore,genre|hardrock,genre|hardstyle,genre|heavymetal,genre|hiphop,genre|honkytonk,genre|house,genre|idm,genre|indian,genre|indie,genre|indiepop,genre|industrial,genre|iranian,genre|jazz,genre|jdance,genre|jidol,genre|jpop,genre|jrock,genre|kids,genre|kpop,genre|latin,genre|latino,genre|malay,genre|mandopop,genre|metal,genre|metalcore,genre|minimaltechno,genre|mpb,genre|newage,genre|opera,genre|pagode,genre|party,genre|piano,genre|pop,genre|popfilm,genre|powerpop,genre|progressivehouse,genre|psychrock,genre|punk,genre|punkrock,genre|reggae,genre|reggaeton,genre|rnb,genre|rock,genre|rockabilly,genre|rocknroll,genre|romance,genre|sad,genre|salsa,genre|samba,genre|sertanejo,genre|showtunes,genre|singersongwriter,genre|ska,genre|sleep,genre|soul,genre|spanish,genre|study,genre|swedish,genre|synthpop,genre|tango,genre|techno,genre|trance,genre|triphop,genre|turkish,genre|worldmusic,danceability,energy,speechiness,acousticness,liveness,valence,tempo,popularity,key|0,key|1,key|2,key|3,key|4,key|5,key|6,key|7,key|8,key|9,key|10,key|11,mode|0,mode|1,track_id
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.137259,0.09220,0.029637,0.006466,0.07160,0.143719,0.072249,0.146,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,5SuOikwiRyPMVoIQDJUgSV
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.085279,0.03320,0.015813,0.185542,0.02020,0.053668,0.063679,0.110,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,4qPNDBW1i3p13qLCt0Ki3A
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.088934,0.07180,0.011544,0.042169,0.02340,0.024121,0.062729,0.114,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,1iJBSr7s7jYXzM8EGcbK5b
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.054010,0.01192

# Model